Classification Algorithms
- Random Forest
- Naive Bayes Classifier
- Decisioin Tree Classifier

In [1]:
# import libraries
import findspark
findspark.init()
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from pyspark.sql.functions import abs
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit 

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark.sparkContext.version)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 15:00:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.2.0


### Discover and visualise the data

In [9]:
# read data from csv file
df = spark.read.csv("/Users/viventan/Desktop/CSCI 316 Big Data Mining Techniques and Implementation/group ass 2/covtype.data", inferSchema=True)

# define the header for each columns
df = df.toDF("Elevation","Aspect","Slope","horHydro","verHydro","horRoad","9am",\
          "Noon","3pm","horFire","wildAr_1","wildAr_2","wildAr_3","wildAr_4",\
          "soilType_1","soilType_2","soilType_3","soilType_4","soilType_5","soilType_6",\
          "soilType_7","soilType_8","soilType_9","soilType_10","soilType_11","soilType_12",\
          "soilType_13","soilType_14","soilType_15","soilType_16","soilType_17","soilType_18",\
          "soilType_19","soilType_20","soilType_21","soilType_22","soilType_23","soilType_24",\
          "soilType_25","soilType_26","soilType_27","soilType_28","soilType_29","soilType_30",\
          "soilType_31","soilType_32","soilType_33","soilType_3a4","soilType_35","soilType_36",\
          "soilType_37","soilType_38","soilType_39","soilType_40","cover")
df

DataFrame[Elevation: int, Aspect: int, Slope: int, horHydro: int, verHydro: int, horRoad: int, 9am: int, Noon: int, 3pm: int, horFire: int, wildAr_1: int, wildAr_2: int, wildAr_3: int, wildAr_4: int, soilType_1: int, soilType_2: int, soilType_3: int, soilType_4: int, soilType_5: int, soilType_6: int, soilType_7: int, soilType_8: int, soilType_9: int, soilType_10: int, soilType_11: int, soilType_12: int, soilType_13: int, soilType_14: int, soilType_15: int, soilType_16: int, soilType_17: int, soilType_18: int, soilType_19: int, soilType_20: int, soilType_21: int, soilType_22: int, soilType_23: int, soilType_24: int, soilType_25: int, soilType_26: int, soilType_27: int, soilType_28: int, soilType_29: int, soilType_30: int, soilType_31: int, soilType_32: int, soilType_33: int, soilType_3a4: int, soilType_35: int, soilType_36: int, soilType_37: int, soilType_38: int, soilType_39: int, soilType_40: int, cover: int]

In [ ]:
df.show()

# Checking PySpark DataFrame schema

In [ ]:
df.printSchema()

In [ ]:
df2 = df.toPandas()

In [ ]:
df2.describe()

In [ ]:
# print out the skewness of each features
print('Skewness of all the features: ')
skewness = df2.skew()
print(skewness)

In [ ]:
# set up the figure size 
plt.figure(figsize=(16,18))

# plot out the diagram to have a clear view of the skewness
sns.barplot(x=df2.skew(), y=skewness.index, palette = 'gist_rainbow_r')

# show the plot
plt.show()

#### Findings:

(1) Many Soil_Type have the extreme right skewed, especially the Soil_Type_15.

(2) The feature 'Hillshade_9am' and 'Hillshade_Noon' have the extreme left skewed which score -1.18 and -1.06 respectively.

In [ ]:
#  returns the correlation matrix of the dataframe
df2.corr()

In [ ]:
# visualize the correlation of all columns in this dataframe
plt.figure(figsize=(15,8))

# using heapmap to plot
sns.heatmap(df2.corr(), annot=True)

# show the plot
plt.show()

That looks abit cranked up for a correlation chart. Hence, we will reduce the number of attributes to just soil_types against the cover_type

In [ ]:
# get and store all the Soil Type features
columns = [cols for cols in df2.columns if str(cols).startswith('Soil')]

# set the figure size
plt.figure(figsize=(12, 12))

# using heapmap to plot
sns.heatmap(df2[columns].corr(), square=True, linewidths=1)

# set the title of the plot
plt.title('Correlation of all Soil Type features')

# show the plot
plt.show()

#### Findings:

(1) There is not any correlation between the soil_types. Hence, we try the remaining columns to see their correlations.

In [ ]:
# get and store all the not Soil Type features
columns = [cols for cols in df2.columns if not str(cols).startswith('Soil')]

# set the figure size
plt.figure(figsize=(12, 12))

# using heapmap to plot
sns.heatmap(df2[columns].corr(), square=True, linewidths=1, annot = True)

# set the title of the plot
plt.title('Correlation of the features')

# show the plot
plt.show()

#### Findings: 

Based on above, we can notice that:

(1) Wilderness_Area_4 has a strong positive correlation with the target (Cover_Type) which gives a 0.32. Hence, it means that from Wilderness_Area_4, we can get a good information for the Cover Type.

(2) Next, Slope is second one that strong correlated with Cover Tpe which gives 0.15.

(3) The most weak correlation is Elevation which has a negative correlation with the target (Cover_Type) and it gives -0.27. Hence, it means that from Elevation, we cannot get any good information about the Cover Type.

(4) The second weak correlation is Wilderness_Area_1 which give a negative correlation -0.2.

In [ ]:
# show the tota number of each cover types in this dataframe
df2.groupby('Cover_Type').size()

In [ ]:
# visualize the count of each 'Cover_Type' in this dataframe
plt.title('Count of each Cover_Type in this dataframe')
sns.set_style('whitegrid')

sns.countplot(x=df2['Cover_Type'], palette = 'gist_rainbow_r')

plt.show()


#### Findings:

(1) The most number of count of Cover Type is Cover_Type 2. Then, second more count of Cover Type is Cover_Type 1. Both informations show that Cover_Type 1 and 2 are the most common cover type in the wildness area.

(2) The least number of count of Cover Type is Cover_Type 4 which is less appear in the wildness area.

In [ ]:
# Visualize the relationship between cover_type and elevation
plt.title('The relationship between Cover_Type and Elevation')
sns.boxplot(data=df2, x='Cover_Type', y='Elevation', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Cover_Type 7 is in the highest elevation level which median falls around 3315. Also, it contains lower outlier falls between 2750 and 3125 and upper outlier which falls between 3500 and 3858.

(2) Comapare with other Cover Type, we notice that most of the Cover_Type 4 are in the lowest elevation level which median falls around 2250 and it does not contain any lower and upper outliers. Its minimum elevation is 2000 and max elevation is around 2600.

(3) Cover Type 3 and Cover Type 4 has no any outliers.

(4) Since all the boxes are very different from each others. Hence, the elevation is a good discriminative feature.

In [ ]:
# Visualize the relationship between cover_type and aspect
plt.title('The relationship between Cover_Type and Aspect')
sns.boxplot(data=df2, x='Cover_Type', y='Aspect', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Most of the Cover Type do not have any lower and upper outliers, except Cover Type 4 contains upper outlier which falls between 275 and 360.

(2) The median of Cover Type 1 and Cover Type 7 is the same at 125.

(3) The minimum and maximum aspect of all Cover Types are the same which is 0 and 360.

(4) Most of the Cover Type's Aspect are in the range between 60 to 240.

(5) This is a good feature since there has least number of outliers which can lead to a more accurate models

In [ ]:
# Visualize the relationship between cover_type and slope
plt.title('The relationship between Cover_Type and Slope')
sns.boxplot(data=df2, x='Cover_Type', y='Slope', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Most of the Cover Types have upper outliers, except Cover Type 4 do not have any outliers in its upper.

(2) Cover Type 2 has the most outliers which compare with other Cover Types and its outlier reaches to 66. It tells us that many unusual observation of Cover Type 2 is more than the maximum usual obseravation of Cover Type 2.

(3) All of the Cover Types have the same minimum slope which is 0.

(4) Cover Type 2 and Cover Type 7 have the same median which slope around 14 which tell us that 50% of their Cover Type are less than or equal to 14.

(5) 25% of the slope of Cover Type 1 and Cover Type 2 are less than or equal to 8.

(6) Cover Type 3 and Cover Type 4 have the same maximun about 46 in usual obeservation data (Cover Type).

In [ ]:
# Visualize the relationship between cover_type and Horizontal_Distance_To_Hydrology
plt.title('The relationship between Cover_Type and Horizontal_Distance_To_Hydrology')
sns.boxplot(data=df2, x='Cover_Type', y='Horizontal_Distance_To_Hydrology', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Most of the Cover Type of their interquartile range (25% ~ 75%) are between 0 to 400, except Cover Type 7.

(2) All the Cover Types have upper outliers, especially Cover Type 1, 2 and 5 where Cover Type 2 reaches 1397.

(3) For Cover Type 4, most of them reach 0, and its maximun value is lower than others as well as it contains the least of upper outliers.

(4) Cover Type 7 has the largest maximun value of the normal observation data which compare to the rest and it reach around 1100.

(5) Most of the Cover Type's maximun value are approximately between 500 and 650.

In [ ]:
# Visualize the relationship between cover_type and Vertical_Distance_To_Hydrology
plt.title('The relationship between Cover_Type and Vertical_Distance_To_Hydrology')
sns.boxplot(data=df2, x='Cover_Type', y='Distance_To_Hydrology', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) 50% of most of the Cover Type of Vertical_Distance_To_Hydrology is less than or equal to 10.

(2) Cover Type 1 and Cover Type 2 contains lots of upper outliers and the largest outlier reaches 601.

(3) Cover Type 2 also has most lower outliers which reaches -173.

(4) Over the all Cover Types, we can see that the minimum Vertical_Distance_To_Hydrology of Cover Type 4 is higher than all of the Cover Type which reaches around -18.

In [ ]:
# Visualize the relationship between cover_type and Horizontal_Distance_To_Roadways
plt.title('The relationship between Cover_Type and Horizontal_Distance_To_Roadways')
sns.boxplot(data=df2, x='Cover_Type', y='Distance_To_Roadways', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Cover Type 5 has a very far outliers from its normal observation data which could affect the machine learning model to do the prediction.

(2) Most of the cover types can be found nearby the roadway in the view of horizontal distance, except Cover_Type 7.

(3) Cover_Type 4 have no any outliers and its maximun value of Horizontal_Distance_To_Roadways is also the lowest one. Based on this information, we can say that most of the Cover Type 4 are nearby the roaadway in the view of Horizontal_Distance_To_Roadways which compare with others.

(4) Other than Cover Type 4, Cover Type 1 and 7 also have no any lower and upper outliers.

In [ ]:
# Visualize the relationship between cover_type and Hillshade_9am
plt.title('The relationship between Cover_Type and Hillshade_9am')
sns.boxplot(data=df2, x='Cover_Type', y='Hillshade_9am', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) As above boxplots, all the Cover Types only have lower outliers, and most of the Cover Types have the extreme outliers far from its maximun point.

(2) The Hillshade_9am has a left skew.

In [ ]:
# Visualize the relationship between cover_type and Hillshade_Noon
plt.title('The relationship between Cover_Type and Hillshade_Noon')
sns.boxplot(data=df2, x='Cover_Type', y='Hillshade_Noon', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) As above boxplots, all the Cover Types only have lower outliers, and Cover Type 2 has a extreme outlier far from its maximun value.

(2) The Hillshade_Noon has a left skew.

In [ ]:
# Visualize the relationship between cover_type and Hillshade_3pm
plt.title('The relationship between Cover_Type and Hillshade_3pm')
sns.boxplot(data=df2, x='Cover_Type', y='Hillshade_3pm', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) Only Cover Type 3, 4 and 5 do not have any lower and upper outliers. Also, their minimun and maximun value are reach to 0 and near 250 respectively.

(2) Hillshade_3pm has a normal skew.

(3) Cover Type 1, 2, 3, 7 have the same median.

(4) Cover Type 1 and 2 have the upper outliers which over 250.

(5) Cover Type 4 and 5's median are lower than the average median of other Cover Types.

In [ ]:
# Visualize the relationship between cover_type and Horizontal_Distance_To_Fire_Points
plt.title('The relationship between Cover_Type and Horizontal_Distance_To_Fire_Points')
sns.boxplot(data=df2, x='Cover_Type', y='Horizontal_Distance_To_Fire_Points', palette='gist_rainbow_r')

plt.show()

#### Findings:

(1) As above, we can see that Cover Type 1 and 2 are mostly the same box shape and same median around 1900 as well as its upper outliers are also the same which almost reach 7173. Next, the maximum value of Horizontal_Distance_To_Fire_Points for both are also near 5000. Besides that, their interquartile range (25% ~ 75%) are almost the same which between 1000 and 2900.

(2) Only Cover Type 4 and 7 do not contain any upper and lower outliers.

(3) For Cover Type 5, it has some upper outliers are very far from its maximun value which would cause the model not that accuracy when doing prediction.

(4) For Cover Type 3 and 4, they mostly have the same median around 800 and their interquartile range (25% ~ 75%) are almost the same which between 500 and 1200.

In [ ]:
# define the columns that want to plot out
areas = ['Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Wilderness_Area_4']
    
# using for loop to generate the graph
for i in range(len(areas)):
    
    # plot i number of graphs
    plt.figure(i, figsize=(12, 7))
 
    # group by each of the cover type and sum each of them
    df2.groupby('Cover_Type')[areas[i]].sum().plot.bar(width = 0.4, color='#ffbf00')
    
    # display the title name
    plt.title(areas[i])

#### Findings:

(1) Most of the Cover Type 1 and 2 are grown in Wilderness_Area_1, Wilderness_Area_2 and Wilderness_Area_3. Only have few of the Cover Type 2 grow in Wilderness_Area_4.

(2) For Cover Type 3, it mainly distributes in Wilderness_Area_4 and got few distribute in Wilderness_Area_3.

(3) Very interesting finding is that for Cover Type 4 which only distribute in Wilderness_Area_4. Other Wilderness_Area do not exist any Cover Type 4 and the amount of Cover Type also less.

(4) For Cover Type 5, it only could be found in Wilderness_Area_1 and Wilderness_Area_3. The amount of Cover Type also less.

(5) Cover Type 6 mainly found in Wilderness_Area_4 and few could be found in Wilderness_Area_3

(7) For Cover Type 7, it could not be found only in Wilderness_Area_4 and the rest have the existence of Cover Type 7.

(8) In Wilderness_Area_3, we could find most of the Cover Types, except the Cover Type 4.

In [ ]:
# get all the columns name
all_columns = df.columns

# eliminate th target column
eliminate_target = len(all_columns)-1

# x-axis is for target column
x = all_columns[eliminate_target]

# y-axis is for the features
y = all_columns[14:eliminate_target]

# plot out the graph
for a in range(0, 40):
    sns.violinplot(data=df2, x=x, y=y[a], palette="muted")
    
    # set the title for each of the graphes
    plt.title('Relationship between Cover_Type and Soil_Type_' + str(a+1))
    plt.show()

#### Findings:

(1) Based on the 40 violinplot graphes above, we can conclude that for Soil_Type 1, 5, 7, 8, 9, 12, 14, 15 18, 21, 22, 25-28 and 35-40 which have at least 4 Cover Types do not have any values.

### Prepare the data for machine learning algorithms

### We first check if the dataset consist of any missing values 

In [ ]:
# Check any missing data for each columns
print(df2.isnull().values.any())

#### Findings:

(1) It gives a False rusult which means that this dataframe does not consist any null values.

In [11]:
# Apply Abs to columns with negative values
df3 = df.withColumn("Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
                "Distance_To_Hydrology", "Distance_To_Roadways",
                "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
                "Horizontal_Distance_To_Fire_Points", "Wilderness_Area_1", "Wilderness_Area_2",
                "Wilderness_Area_3", "Wilderness_Area_4", "Soil_Type_1", 
                "Soil_Type_2", "Soil_Type_3", "Soil_Type_4", "Soil_Type_5", 
                "Soil_Type_6", "Soil_Type_7", "Soil_Type_8", "Soil_Type_9", "Soil_Type_10", 
                "Soil_Type_11", "Soil_Type_12", "Soil_Type_13", "Soil_Type_14", "Soil_Type_15", 
                "Soil_Type_16", "Soil_Type_17", "Soil_Type_18", "Soil_Type_19", "Soil_Type_20", 
                "Soil_Type_21", "Soil_Type_22", "Soil_Type_23", "Soil_Type_24", "Soil_Type_25", 
                "Soil_Type_26", "Soil_Type_27", "Soil_Type_28", "Soil_Type_29", "Soil_Type_30", 
                "Soil_Type_31", "Soil_Type_32", "Soil_Type_33", "Soil_Type_34", "Soil_Type_35", 
                "Soil_Type_36", "Soil_Type_37", "Soil_Type_38", "Soil_Type_39", 
                "Soil_Type_40", "Cover_Type", abs(df.Elevation,Aspect,Slope,horHydro,verHydro,horRoad,9am,\
          Noon,3pm,horFire,wildAr_1,wildAr_2,wildAr_3,wildAr_4,\
          soilType_1,soilType_2,soilType_3,soilType_4,soilType_5,soilType_6,\
          soilType_7,soilType_8,soilType_9,soilType_11,soilType_12",\
          soilType_13,soilType_14,soilType_15,soilType_16,soilType_17,soilType_18",\
          soilType_19,soilType_20,soilType_21,soilType_22,soilType_23,soilType_24",\
          soilType_25,soilType_26,soilType_27,soilType_28,soilType_29,soilType_30",\
          soilType_31,soilType_32,soilType_33,soilType_3a4,soilType_35,soilType_36",\
          soilType_37,soilType_38,soilType_39,soilType_40"))
df3.show(3)

TypeError: Invalid argument, not a string or column: DataFrame[Elevation: int, Aspect: int, Slope: int, horHydro: int, verHydro: int, horRoad: int, 9am: int, Noon: int, 3pm: int, horFire: int, wildAr_1: int, wildAr_2: int, wildAr_3: int, wildAr_4: int, soilType_1: int, soilType_2: int, soilType_3: int, soilType_4: int, soilType_5: int, soilType_6: int, soilType_7: int, soilType_8: int, soilType_9: int, soilType_10: int, soilType_11: int, soilType_12: int, soilType_13: int, soilType_14: int, soilType_15: int, soilType_16: int, soilType_17: int, soilType_18: int, soilType_19: int, soilType_20: int, soilType_21: int, soilType_22: int, soilType_23: int, soilType_24: int, soilType_25: int, soilType_26: int, soilType_27: int, soilType_28: int, soilType_29: int, soilType_30: int, soilType_31: int, soilType_32: int, soilType_33: int, soilType_3a4: int, soilType_35: int, soilType_36: int, soilType_37: int, soilType_38: int, soilType_39: int, soilType_40: int, cover: int] of type <class 'pyspark

In [ ]:
# Drop rows with negative values
df = df4.drop("Distance_To_Hydrology","Distance_To_Roadways")

## Feature preprocessing

In [ ]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler(\
inputCols=["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
                "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
                "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
                "Horizontal_Distance_To_Fire_Points", "Wilderness_Area_1", "Wilderness_Area_2",
                "Wilderness_Area_3", "Wilderness_Area_4", "Soil_Type_1", 
                "Soil_Type_2", "Soil_Type_3", "Soil_Type_4", "Soil_Type_5", 
                "Soil_Type_6", "Soil_Type_7", "Soil_Type_8", "Soil_Type_9", "Soil_Type_10", 
                "Soil_Type_11", "Soil_Type_12", "Soil_Type_13", "Soil_Type_14", "Soil_Type_15", 
                "Soil_Type_16", "Soil_Type_17", "Soil_Type_18", "Soil_Type_19", "Soil_Type_20", 
                "Soil_Type_21", "Soil_Type_22", "Soil_Type_23", "Soil_Type_24", "Soil_Type_25", 
                "Soil_Type_26", "Soil_Type_27", "Soil_Type_28", "Soil_Type_29", "Soil_Type_30", 
                "Soil_Type_31", "Soil_Type_32", "Soil_Type_33", "Soil_Type_34", "Soil_Type_35", 
                "Soil_Type_36", "Soil_Type_37", "Soil_Type_38", "Soil_Type_39", 
                "Soil_Type_40"],\
outputCol="features")


In [ ]:
l_indexer = StringIndexer(inputCol="Cover_Type", outputCol="label")

## Stratified sampling

In [ ]:
df.groupBy("Cover_Type").count().orderBy("Cover_Type").show()

In [ ]:
# Taking 2/3 cover-type into training set
train = df.sampleBy("Cover_Type", fractions={1:2/3,2:2/3,3:2/3,4:2/3,5:2/3,6:2/3,7:2/3}, seed=0)
# Subtracting 'train' from original 'data' to get test set 
test = df.subtract(train)

In [ ]:
train.groupBy("Cover_Type").count().orderBy("Cover_Type").show()

In [ ]:
test.groupBy("Cover_Type").count().orderBy("Cover_Type").show()

# 3 Clssification Models

## (1) Decision Tree classifier

- Strength 
   - i.  Easy to interpret
   - ii. Handle categorical features
   - iii. Able to capture non-linearities and feature interactions

- Weaknesses
   - i. A small change in the data can cause a large change in the structure of the decision tree causing instability

   - ii. Calculations can be more complex compared to other algorithms
   - iii. longer training time

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

### Build the pipeline

- Pipeline is to combine multiple steps into a single workflow that can be easily run.
- Build a pipeline combining all of our feature engineering and modeling steps

In [ ]:
from pyspark.ml import Pipeline
 
# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[l_indexer, vector_assembler, dt])
 
# Define the pipeline model.
pipelineModel = pipeline.fit(train)
 
# Apply the pipeline model to the test dataset.
predictions = pipelineModel.transform(test)

In [ ]:
# Display the predictions from the model
predictions.select("prediction", "label").show(5)

### Area under ROC curve
- As the area under an ROC curve is a measure of the usefulness of a test in general, where a greater area means a more useful test

In [ ]:
# evalute the area under the ROC curve 
BCevaluator = BinaryClassificationEvaluator()\
              .setMetricName("areaUnderROC")\
              .setRawPredictionCol("prediction")\
              .setLabelCol("label")
print(f"Area under ROC curve : {BCevaluator.evaluate(predictions)}")

# evalute the accuracy
MCevaluator = MulticlassClassificationEvaluator(\
              labelCol="label", predictionCol="prediction",\
              metricName="accuracy")
print(f"Accuracy: {MCevaluator.evaluate(predictions)}")

#### Findings:

 (1) The area under ROC curve is 77.01% and it is considered acceptable.
 
 (2) Accuracy before tuning is 70.22%.

## Tuning the Decisioin Tree Classifier

In [ ]:
# define a grid search over a set of model hyperparameters.
params = ParamGridBuilder().addGrid(dt.maxDepth, [10,12,14]).addGrid(dt.maxBins, [7,8]).build()

In [ ]:
# TrainValidationSplit
tvs = TrainValidationSplit().setTrainRatio(2/3).setEstimatorParamMaps(params).setEstimator(pipeline).setEvaluator(BCevaluator)

cvModel = tvs.fit(train)
# Use the model identified by the TrainValidationSplit to make predictions on the test dataset
cvPredDF = cvModel.transform(test)


In [ ]:
# Evaluate the model's performance based on area under the ROC curve and accuracy 
print(f"Area under ROC curve: {BCevaluator.evaluate(cvPredDF)}")
print(f"Accuracy: {MCevaluator.evaluate(cvPredDF)}")

#### Findings:

After tuning the model:
(1) First, we can see that the area under ROC curve becomes 85.40% which compare with the area under ROC curve 77.01% (without tuning) has an significant improvement of 8.39% and this is considered excellent.

(2) Next, we can see that the accuracy becomes 82.15% which compare with the accuracy 70.22% (without tuning) has an significant improvement of 11.93%.

(3)  Furthermore, We selected 3 options for the maxdepth parameter and 2 options for maxBIns parameters then using the TrainValidationSplit to make the predictions and it gives us a better accuracy.

## (2) Random Forest

- Strength 
   - i.  Works well with both categorical and continuous values
   - ii. Comparatively less impacted by noise
   - iii. Automatically handle missing values

- Weaknesses
   - i. Requires much computational power and resources as it builds numerous trees to combine their outputs. 

   - ii. Requires much more time to train as compared to decision trees as it generates a lot of trees and makes decision on the majority of votes.

In [ ]:
rf = RandomForestClassifier(labelCol="label",\
featuresCol="features", numTrees=10)

### Build the pipeline

In [ ]:
from pyspark.ml import Pipeline
 
# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[l_indexer, vector_assembler, rf])
 
# Define the pipeline model.
pipelineModel = pipeline.fit(train)
 
# Apply the pipeline model to the test dataset.
predictions = pipelineModel.transform(test)

In [ ]:
# Display the predictions from the model
predictions.select("prediction", "label").show(5)

In [ ]:
# evalute the area under the ROC curve 
BCevaluator = BinaryClassificationEvaluator()\
              .setMetricName("areaUnderROC")\
              .setRawPredictionCol("prediction")\
              .setLabelCol("label")
print(f"Area under ROC curve : {BCevaluator.evaluate(predictions)}")

# evalute the accuracy
MCevaluator = MulticlassClassificationEvaluator(\
              labelCol="label", predictionCol="prediction",\
              metricName="accuracy")
print(f"Accuracy: {MCevaluator.evaluate(predictions)}")

#### Findings:

 (1) The area under ROC curve is 74.26% and it is considered acceptable.
 
 (2) Accuracy before tuning is 67.30%.

### Tuning the Random Forest model

In [ ]:
# define a grid search over a set of model hyperparameters.
params = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
               .addGrid(rf.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
               .addGrid(rf.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
               .addGrid(rf.numTrees, [5, 20, 50])
             .build())

In [ ]:
# TrainValidationSplit
tvs = TrainValidationSplit().setTrainRatio(2/3).setEstimatorParamMaps(params).setEstimator(pipeline).setEvaluator(BCevaluator)

cvModel = tvs.fit(train)
# Use the model identified by the TrainValidationSplit to make predictions on the test dataset
cvPredDF = cvModel.transform(test)

In [ ]:
# Evaluate the model's performance based on area under the ROC curve and accuracy 
print(f"Area under ROC curve: {BCevaluator.evaluate(cvPredDF)}")
print(f"Accuracy: {MCevaluator.evaluate(cvPredDF)}")

#### Findings:

After tuning the model:

(1) First, we can see that the area under ROC curve becomes 78.86% which compare with the area under ROC 
curve 74.26% (without tuning) has an improvement of 4.6% and this is considered quite good.

(2) Next, we can see that the accuracy becomes 73.17% which compare with the accuracy 67.30% (without tuning) 
has an improvement of 5.87%.

(3)  Furthermore, We selected 3 options for the maxdepth parameter,3 options for maxBIns parameters and 
3 options for numTrees then using the TrainValidationSplit to make the predictions and it gives us a better 
accuracy.

## (3) Naive Bayes Classifier

- Strengths:
    - i. Able to handle an extremely large number of features
    - ii. Relatively simple to implement and tune the model.
    
- Weaknessess:
    - i. Does not solve negative values in the dataset. Hence, we apply abs() function to columns containing negative values in the dataset.
    - ii. If categorical variable has a category in test data set, which was not observed in training data set, then model will assign a 0 (zero) probability and will be unable to make a prediction.
    

In [ ]:
# create the nb and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

### Build the pipeline

In [ ]:
# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[l_indexer, vector_assembler, nb])
 
# Define the pipeline model.
pipelineModel = pipeline.fit(train)
 
# Apply the pipeline model to the test dataset.
predictions = pipelineModel.transform(test)

In [ ]:
predictions.select("prediction", "label").show(5)

In [ ]:
BCevaluator = BinaryClassificationEvaluator()\
          .setMetricName("areaUnderROC")\
          .setRawPredictionCol("prediction")\
          .setLabelCol("label")
print(f"Area under ROC curve : {BCevaluator.evaluate(predictions)}")

MCevaluator = MulticlassClassificationEvaluator(\
labelCol="label", predictionCol="prediction",\
metricName="accuracy")
print(f"Accuracy: {MCevaluator.evaluate(predictions)}")

#### Findings:

 (1) The area under ROC curve is 53.01% and it is considered acceptable.
 
 (2) Accuracy before tuning is 27.14%.

### Tuning Naive Bayes Classifier

In [ ]:
params = ParamGridBuilder().addGrid(nb.smoothing, [0.5, 0.7, 0.9, 1.1, 1.2, 1.0]).build()

In [ ]:
# TrainValidationSplit
tvs = TrainValidationSplit().setTrainRatio(2/3).setEstimatorParamMaps(params).setEstimator(pipeline).setEvaluator(MCevaluator)

cvModel = tvs.fit(train)
# Use the model identified by the TrainValidationSplit to make predictions on the test dataset
cvPredDF = cvModel.transform(test)

In [ ]:
# Evaluate the model's performance based on area under the ROC curve and accuracy 
print(f"Area under ROC curve: {BCevaluator.evaluate(cvPredDF)}")
print(f"Accuracy: {MCevaluator.evaluate(cvPredDF)}")

#### Findings:

After tuning the model:
(1) First, we can see that the area under ROC curve becomes 53.00% which compare with the area under ROC curve 53.01% (without tuning) has only an improvement of 0.01%.

(2) Next, we can see that the accuracy becomes 27.16% which compare with the accuracy 27.14% (without tuning) has an improvement of 0.02%.

(3) Furthermore, We selected 6 options for var smoothing then using the TrainValidationSplit to make the predictions and the accuracy does not improved much.

#### Finding of comparing these 3 classification models above:
    
In this task, we have used Decisioin Tree Classifier,Random Forest model and Naive Bayes Classifier model to solve the classification problem.

Among these 3 models, Random Forest model and Decisioin Tree Classifier model work quite well in this task. Decisioin Tree Classifier gets an approximately 70% and 82% before and after tuning the models. Also, Random Forest model gets an approiximately 67% and 73% before and after tuning the model.

However, Naive Bayes Classifier model gives a poor result in solving the classification problem of this task even after tuning the model.

## User-Defined Transform functionality

In [ ]:
def categorizer(Elevation):
  if Elevation < 2500:
    return 1
  elif Elevation < 3000:
    return 2
  else :
    return 3


In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf,col
  
elevation_level = udf(lambda Elevation: categorizer(Elevation), IntegerType())
df_new = df.withColumn('Elevation_lvl', elevation_level(col('Elevation')))
df_new.show()

#### Findings:

(1) We generate a new feature called 'Elevation_lvl' since the feature 'Elevation' contains many different values and do not have a specific range for the models to do prediction.

(2) The new feature 'Elevation_lvl' has categorize the Elevation values into 3 catrgories which has 1, 2 and 3 so that we can based on this new feature to see the elevation each Cover Type belong to low, median or high. But in our case, we use number instead of string to label the category.


In [ ]:
df_new.groupBy("Elevation_lvl","Cover_Type").count().orderBy("Elevation_lvl").show()

#### Findings:

(1) Based on the counts above, we can see that most of the Cover Types are in Elevation_Lvl 1 and 2, except Cover Type 7 which does not appear any in Elevation_Level 1. Also, the elevation level of Cover Type 1,2 and 5 are average distribution which appear in all 3 elevation level 1, 2 and 3. 

(2) Next, we notice that only Cover Type 1,2,5 and 7 have in Elevatioin_Level 3. Hence, we can conclude that Cover Type 3, 4, and 6 do not grow in the elevation level 3 based on this dataset.